In [1]:
# Imports
from pandas_datareader import data as pdr
from yahoo_fin import stock_info as si
from pandas import ExcelWriter
import yfinance as yf
import pandas as pd
import datetime
import time
yf.pdr_override()
from pathlib import Path
import numpy as np

In [2]:
# scan a list of stocks from a csv file of Investors Business Daily recommendations
csv_path = Path("Dayakar_List_3.csv")
ibd_df = pd.read_csv(csv_path)

In [3]:
ibd_df.tail()

,Symbol
29,SNOW
30,TEAM
31,TMO
32,ULTA
33,UPST


In [4]:
# Variables, tickers from the IBD list
#tickers = si.tickers_sp500()
tickers = ibd_df["Symbol"]
tickers = [item.replace(".", "-") for item in tickers] # Yahoo Finance uses dashes instead of dots
index_name = 'IBD' # S&P 500
exportList = pd.DataFrame(columns=['Stock', "RS_Rating", "50 Day MA", "150 Day Ma", "200 Day MA", "52 Week Low", "52 week High"])
returns_multiples = []

In [5]:
#Index from the list above Returns
index_df = pdr.get_data_yahoo(index_name)
index_df['Percent Change'] = index_df['Adj Close'].pct_change()
index_return = (index_df['Percent Change'] + 1).cumprod()[-1]

[*********************100%***********************]  1 of 1 completed


In [6]:
# parameter setup (default values in the original indicator)
#These are the constants to be used for the moving averages, the bollinger bands and the keltner channels
#ultimately these will be needed for the squeeze and other calculations
import contextlib
length = 20
mult = 2
length_KC = 21
mult_KC = 2

In [14]:
period = input('enter period of data eg 1y or 2y:')
interval = input('enter candle interval eg 1d, 1wk, 1mo, 1h')
file_path_d = f'scanoutput{interval}.txt'
with open(file_path_d, "w") as o:
    for ticker in tickers:
        # Download historical data as CSV for each stock (makes the process faster)
        stock = yf.Ticker(ticker)
        df = stock.history(period=period, interval=interval)
        df.index.duplicated().sum()
        df = df.loc[~df.index.duplicated(keep='first')]
        df.dropna(inplace=True)
        df.to_csv(f'{ticker}_{period}_{interval}.csv')    
        m_avg = df['Close'].ewm(span=length).mean()
        df['Moving average'] = m_avg
        # standard deviation
        m_std = df['Close'].rolling(window=length).std()
        # upper Bollinger Bands
        df['upper_BB'] = m_avg + mult * m_std
        # lower Bollinger Bands 
        df['lower_BB'] = m_avg - mult * m_std
        df['bol_percent_b'] = ((df['Close'] - df['lower_BB'])/(df['upper_BB'] - df['lower_BB']))*100
        # calculate Keltner Channel which is a measure of the average true range.  In this case the channels are set to twice the average true range
        # first we need to calculate True Range
        df['tr0'] = abs(df["High"] - df["Low"])
        df['tr1'] = abs(df["High"] - df["Close"].shift())
        df['tr2'] = abs(df["Low"] - df["Close"].shift())
        df['tr'] = df[['tr0', 'tr1', 'tr2']].max(axis=1)
        # moving average of the TR or average true range
        range_ma = df['tr'].ewm(span=length_KC).mean()
        # upper Keltner Channel
        df['upper_KC'] = m_avg + range_ma * mult_KC
        # lower Keltner Channel
        df['lower_KC'] = m_avg - range_ma * mult_KC

        df['squeeze_on'] = (df['lower_BB'] > df['lower_KC']) & (df['upper_BB'] < df['upper_KC'])
        df['squeeze_off'] = (df['lower_BB'] < df['lower_KC']) & (df['upper_BB'] > df['upper_KC'])

        highest = df['High'].rolling(window = length_KC).max()
        lowest = df['Low'].rolling(window = length_KC).min()
        m1 = (highest + lowest) / 2
        df['value'] = (df['Close'] - (m1 + m_avg)/2)
        fit_y = np.array(range(0,length_KC))
        df['value'] = df['value'].rolling(window = length_KC).apply(lambda x : np.polyfit(fit_y, x, 1)[0] * (length_KC-1) +
        np.polyfit(fit_y, x, 1)[1], raw=True)

        df[['squeeze_on', 'squeeze_off']] = (df[['squeeze_on', 'squeeze_off']] == True).astype(float)

        df["pct_change"] = df['Close'].pct_change()*100

        conditions =  [
            (df['pct_change'] > 0),
            (df['pct_change'] < 0)
        ]

        label = [1, 0]
        df['profitable?'] = np.select(conditions, label) 

        # entry point for long position:
        # 1. black cross becomes gray (the squeeze is released)
        long_cond1 = (df['squeeze_off'][-2] == 0) | (df['squeeze_off'][-1] == 1) 
        # 2. bar value is positive => the bar is light green
        long_cond2 = df['value'][-1] > 0

        enter_long = long_cond1 and long_cond2
        # entry point for short position:
        # 1. black cross becomes gray (the squeeze is released)
        short_cond1 = (df['squeeze_off'][-2] == 0) | (df['squeeze_off'][-1] == 1) 
        # 2. bar value is negative => the bar is light red 
        short_cond2 = df['value'][-1] < 0
        enter_short = short_cond1 and short_cond2

        colors = []
        for ind, val in enumerate(df['value']):
            if val >= 0:
                color = 'blue'
                if val > df['value'][ind-1]:
                    color = 'cyan'

            else:
                color = 'yellow'
                if val < df['value'][ind-1]:
                    color='red'        
            colors.append(color)    
        df['colors'] = colors
        color_value = df['colors'][-1]
        squeeze_value = df['squeeze_on'][-1]
        
        m_avg_08 = df['Close'].ewm(span=8).mean()
        m_avg_21 = df['Close'].ewm(span=21).mean()
        m_avg_34 = df['Close'].ewm(span=34).mean()
        m_avg_55 = df['Close'].ewm(span=55).mean()
        m_avg_89 = df['Close'].ewm(span=89).mean()

        df['m_avg_08'] = m_avg_08
        df['m_avg_21'] = m_avg_21
        df['m_avg_34'] = m_avg_34
        df['m_avg_55'] = m_avg_55
        df['m_avg_89'] = m_avg_89

        df['tr0'] = abs(df["High"] - df["Low"])
        df['tr1'] = abs(df["High"] - df["Close"].shift())
        df['tr2'] = abs(df["Low"] - df["Close"].shift())
        df['tr'] = df[['tr0', 'tr1', 'tr2']].max(axis=1)
        # moving average of the TR or average true range
        range_ma = df['tr'].ewm(span=length_KC).mean()
        # upper Keltner Channel
        df['upper_KC'] = m_avg + range_ma * mult_KC
        # lower Keltner Channel
        df['lower_KC'] = m_avg - range_ma * mult_KC

        plus_dm = df["High"].diff()
        minus_dm = df["Low"].diff()
        plus_dm[plus_dm < 0] = 0
        minus_dm[minus_dm > 0] = 0
        
        df['atr'] = df['tr'].ewm(span=14).mean()
        df['plus_di'] = 100 * (plus_dm.ewm(alpha = 1/14).mean() / df['atr'])
        df['minus_di'] = abs(100 * (minus_dm.ewm(alpha = 1/14).mean() / df['atr']))
        dx = (abs(df['plus_di'] - df['minus_di']) / abs(df['plus_di'] + df['minus_di'])) * 100
        adx = ((dx.shift(1) * (14 - 1)) + dx) / 14
        df['adx'] = adx.ewm(alpha = 1/14).mean() #ADX>20 is trending ADX<20 not trending

        df['ADX_trend'] = 20
        df['percent_b_high'] = 90
        df['percent_b_low'] = 10

        with contextlib.redirect_stdout(o):
            print (f'Ticker: {ticker}; Squeeze_scan\n {squeeze_value},{color_value}')
            if df['bol_percent_b'][-1] >= df['percent_b_high'][-1]:
                print('stock is very close to the upper bollinger, watch the candles for bearishness and possible news and either take profits or start hedging.  Also watch for squeeze')
            elif  df['bol_percent_b'][-1] <= df['percent_b_low'][-1]:
                print('stock is very close to the lower bollinger or even below it, watch the candles for bullishness and possible news.  Cover shorts if any or start looking to buy if the market is conducive.  Watch for squeeze as well')
            else:
                print('stock is between the bollingers at the moment, it is neutral')

            if df['Close'][-1] >= 0.999*df['upper_KC'][-1]:
                print(f'{ticker} is very close to the upper keltner channel, watch the candles for bearishness and possible news and either take profits or start hedging')
            else:
                print(f'{ticker} has room to run for the moment for the keltners')
                
            if (df['m_avg_08'][-1] > df['m_avg_21'][-1] > df['m_avg_34'][-1] > df['m_avg_55'][-1] > df['m_avg_89'][-1]):
                print (f'Ticker: {ticker}; Stacked EMA\n is in the positive direction')
            elif (df['m_avg_08'][-1] < df['m_avg_21'][-1] < df['m_avg_34'][-1] < df['m_avg_55'][-1] < df['m_avg_89'][-1]):
                print (f'Ticker: {ticker}; Stacked EMA\n is in the negative direction')
            else:
                print (f'Ticker: {ticker}; Stacked EMA\n is neutral')
        
            if (df['adx'][-1] > df['ADX_trend'][-1]) and (df['plus_di'][-1] > df['minus_di'][-1]):
                print (f'Ticker: {ticker}; Directional trend is positive\n')
            elif (df['adx'][-1] > df['ADX_trend'][-1]) and (df['plus_di'][-1] < df['minus_di'][-1]):
                print (f'Ticker: {ticker}; Directional trend is negative\n')
            elif (df['adx'][-1] < df['ADX_trend'][-1]):
                print (f'Ticker: {ticker}; Directional trend is neutral or sideways\n')
        time.sleep(1)

enter period of data eg 1y or 2y: 10y
enter candle interval eg 1d, 1wk, 1mo, 1h 1mo


In [15]:
  
# Python code to illustrate Sending mail with attachments 
# from your Gmail account  
  
# libraries to be imported 
import smtplib 
from email.mime.multipart import MIMEMultipart 
from email.mime.text import MIMEText 
from email.mime.base import MIMEBase 
from email import encoders 
   
fromaddr = "akkrishn1@gmail.com"
toaddr = "dayakar1471@gmail.com"
   
# instance of MIMEMultipart 
msg = MIMEMultipart() 
  
# storing the senders email address   
msg['From'] = fromaddr 
  
# storing the receivers email address  
msg['To'] = toaddr 
  
# storing the subject  
msg['Subject'] = f"Scan output {period} {interval}"
  
# string to store the body of the mail 
body = f"Here is the scanner output for Dayakar List 2 {period} {interval}"
  
# attach the body with the msg instance 
msg.attach(MIMEText(body, 'plain')) 
  
# open the file to be sent  
filename = f"scanoutput{interval}.txt"
attachment = open(f"C:/Users/akkrishn/SqueezeMeBabyOneMoreTime/Dayakar List/scanoutput{interval}.txt", "rb") 
  
# instance of MIMEBase and named as p 
p = MIMEBase('application', 'octet-stream') 
  
# To change the payload into encoded form 
p.set_payload((attachment).read()) 
  
# encode into base64 
encoders.encode_base64(p) 
   
p.add_header('Content-Disposition', "attachment; filename= %s" % filename) 
  
# attach the instance 'p' to instance 'msg' 
msg.attach(p) 
  
# creates SMTP session 
s = smtplib.SMTP('smtp.gmail.com', 587) 
  
# start TLS for security 
s.starttls() 
  
# Authentication 
s.login(fromaddr, "Kr15hn40709") 
  
# Converts the Multipart msg into a string 
text = msg.as_string() 
  
# sending the mail 
s.sendmail(fromaddr, toaddr, text) 
  
# terminating the session 
s.quit() 

(221, b'2.0.0 closing connection gm2sm3971741pjb.21 - gsmtp')